In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, json, requests, pickle
from scipy.stats import skew
from shapely.geometry import Point,Polygon,MultiPoint,MultiPolygon
from scipy.stats import ttest_ind, f_oneway, lognorm, levy, skew, chisquare
#import scipy.stats as st
from sklearn.preprocessing import normalize, scale
from tabulate import tabulate #pretty print of tables. source: http://txt.arboreus.com/2013/03/13/pretty-print-tables-in-python.html
from shapely.geometry import Point,Polygon,MultiPoint

%matplotlib inline

ModuleNotFoundError: No module named 'shapely'

based on New York green taxi record, do exploratory data analysis and then making prediction model 
about how much tips will driver get per trip

Load in data and take a first glance

In [35]:
if os.path.exists("green_tripdata_2015-09.csv"):
    nytaxi = pd.read_csv("green_tripdata_2015-09.csv")
else:
    url = "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2015-09.csv"
    nytaxi = pd.read_csv(url)
    nytaxi.to_csv(url.split('/')[-1])
print("Number of obs:", nytaxi.shape[0])
print("Number of features:", nytaxi.shape[1])
print(nytaxi.head())

Number of obs: 1494926
Number of features: 21
   VendorID lpep_pickup_datetime Lpep_dropoff_datetime Store_and_fwd_flag  \
0         2  2015-09-01 00:02:34   2015-09-01 00:02:38                  N   
1         2  2015-09-01 00:04:20   2015-09-01 00:04:24                  N   
2         2  2015-09-01 00:01:50   2015-09-01 00:04:24                  N   
3         2  2015-09-01 00:02:36   2015-09-01 00:06:42                  N   
4         2  2015-09-01 00:00:14   2015-09-01 00:04:20                  N   

   RateCodeID  Pickup_longitude  Pickup_latitude  Dropoff_longitude  \
0           5        -73.979485        40.684956         -73.979431   
1           5        -74.010796        40.912216         -74.010780   
2           1        -73.921410        40.766708         -73.914413   
3           1        -73.921387        40.766678         -73.931427   
4           1        -73.955482        40.714046         -73.944412   

   Dropoff_latitude  Passenger_count     ...      Fare_amount  E

In [26]:
nytaxi = pd.read_csv("green_tripdata_2015-09.csv")
#take a glance at our data
print(nytaxi.head())
print(list(nytaxi))

   VendorID lpep_pickup_datetime Lpep_dropoff_datetime Store_and_fwd_flag  \
0         2  2015-09-01 00:02:34   2015-09-01 00:02:38                  N   
1         2  2015-09-01 00:04:20   2015-09-01 00:04:24                  N   
2         2  2015-09-01 00:01:50   2015-09-01 00:04:24                  N   
3         2  2015-09-01 00:02:36   2015-09-01 00:06:42                  N   
4         2  2015-09-01 00:00:14   2015-09-01 00:04:20                  N   

   RateCodeID  Pickup_longitude  Pickup_latitude  Dropoff_longitude  \
0           5        -73.979485        40.684956         -73.979431   
1           5        -74.010796        40.912216         -74.010780   
2           1        -73.921410        40.766708         -73.914413   
3           1        -73.921387        40.766678         -73.931427   
4           1        -73.955482        40.714046         -73.944412   

   Dropoff_latitude  Passenger_count     ...      Fare_amount  Extra  MTA_tax  \
0         40.685020          

In [30]:
#the last variable is really annoying, with "Trip_type "(there is a blank at the end!), rename it
nytaxi.rename(columns ={'Trip_type ':'Trip_type'}, inplace = True)
list(nytaxi)

['VendorID',
 'lpep_pickup_datetime',
 'Lpep_dropoff_datetime',
 'Store_and_fwd_flag',
 'RateCodeID',
 'Pickup_longitude',
 'Pickup_latitude',
 'Dropoff_longitude',
 'Dropoff_latitude',
 'Passenger_count',
 'Trip_distance',
 'Fare_amount',
 'Extra',
 'MTA_tax',
 'Tip_amount',
 'Tolls_amount',
 'Ehail_fee',
 'improvement_surcharge',
 'Total_amount',
 'Payment_type',
 'Trip_type']

In [8]:
#see some numbers and types of our data
print("Number of obs:", nytaxi.shape[0])
print("Number of features:", nytaxi.shape[1])
print(nytaxi.info())
print(nytaxi.describe())
'''
Actually, from the head of dataset, we can see the features:
lpep_pickup_datetime, lpep_dropoff_datetime is DATE
Store_and_fwd_flag is Boolean
VendorID, RateCodeID, Passenger_count, Payment_type, Trip_type are discrete variables
'''

Number of obs: 1494926
Number of features: 21
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1494926 entries, 0 to 1494925
Data columns (total 21 columns):
VendorID                 1494926 non-null int64
lpep_pickup_datetime     1494926 non-null object
Lpep_dropoff_datetime    1494926 non-null object
Store_and_fwd_flag       1494926 non-null object
RateCodeID               1494926 non-null int64
Pickup_longitude         1494926 non-null float64
Pickup_latitude          1494926 non-null float64
Dropoff_longitude        1494926 non-null float64
Dropoff_latitude         1494926 non-null float64
Passenger_count          1494926 non-null int64
Trip_distance            1494926 non-null float64
Fare_amount              1494926 non-null float64
Extra                    1494926 non-null float64
MTA_tax                  1494926 non-null float64
Tip_amount               1494926 non-null float64
Tolls_amount             1494926 non-null float64
Ehail_fee                0 non-null float64
improv

In [31]:
print(nytaxi["VendorID"].value_counts())
print(nytaxi["RateCodeID"].value_counts())
print(nytaxi["Passenger_count"].value_counts())
print(nytaxi["Payment_type"].value_counts())
print(nytaxi["Trip_type"].value_counts())

2    1169099
1     325827
Name: VendorID, dtype: int64
1     1454464
5       33943
2        4435
3        1117
4         925
6          36
99          6
Name: RateCodeID, dtype: int64
1    1255832
2     115014
5      57907
3      33812
6      22091
4       9769
0        440
8         30
9         16
7         15
Name: Passenger_count, dtype: int64
2    783699
1    701287
3      5498
4      4368
5        74
Name: Payment_type, dtype: int64
Trip_type    1494926
Name: Trip_type, dtype: int64


In [48]:
#take a look at the distribution of those variables
fig, ax = plt.subplots(1,2, figsize = (10,4))
nytaxi.Trip_distance.hist(bins = 30, ax = ax[0])
ax[0].set_xlabel('Trip Distance (miles)')
ax[0].set_ylabel('Count')
ax[0].set_yscale('log')
ax[0].set_title('Histogram of Trip Distance with outliers included')
#now we can see there are some outliers with extremely long trip distance
#we may mainly focus on majority data, so drop those which are outside of 3sigma from the mean
#plot histogram again
distance = nytaxi.Trip_distance
distance_d = distance[((distance-distance.median()).abs() > 3* distance.std())]
print(distance_d.shape[0], distance.shape[0])
#if use mean-3*sd, there is only 29192/1494926 left.. try median there is 38204/1494926 left 
distance_d.hist(bins = 30, ax=ax[1])
ax[1].set_xlabel('Trip Distance (miles)')
ax[1].set_ylabel('Count')
ax[1].set_title('A. Histogram of Trip Distance (without outliers)')

#add a fitness line(lognormal) Use the mean of trip distance as the scale parameter
scatter,loc,mean = lognorm.fit(nytaxi.Trip_distance.values,
                               scale=nytaxi.Trip_distance.mean(),
                               loc=0)
pdf_fitted = lognorm.pdf(np.arange(0,12,.1),scatter,loc,mean)
ax[1].plot(np.arange(0,12,.1),600000*pdf_fitted,'r') 
ax[1].legend(['data','lognormal fit'])
plt.show()


38204 1494926


NameError: name 'lognorm' is not defined

38204 1494926


"\n.hist(bins = 30, ax=ax[1])\nax[1].set_xlabel('Trip Distance (miles)')\nax[1].set_ylabel('Count')\nax[1].set_title('A. Histogram of Trip Distance (without outliers)')\n"